In [28]:
import numpy as np

class QM9NpzWriter():
    def __init__(self, npz_path: str, standardize_targets: bool = False):
        '''
        code readapted from: https://docs.dgl.ai/en/0.8.x/_modules/dgl/data/qm9.html
        urls: https://data.dgl.ai/dataset/qm9_eV.npz or https://github.com/gasteigerjo/dimenet/blob/master/data/qm9_eV.npz
        130,831 molecules with 12 regression targets

        available targets:
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | Keys   | Property                         | Description                                                                       | Unit                                        |
        +========+==================================+===================================================================================+=============================================+
        | mu     | :math:`\mu`                      | Dipole moment                                                                     | :math:`\textrm{D}`                          |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | alpha  | :math:`\alpha`                   | Isotropic polarizability                                                          | :math:`{a_0}^3`                             |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | homo   | :math:`\epsilon_{\textrm{HOMO}}` | Highest occupied molecular orbital energy                                         | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | lumo   | :math:`\epsilon_{\textrm{LUMO}}` | Lowest unoccupied molecular orbital energy                                        | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | gap    | :math:`\Delta \epsilon`          | Gap between :math:`\epsilon_{\textrm{HOMO}}` and :math:`\epsilon_{\textrm{LUMO}}` | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | r2     | :math:`\langle R^2 \rangle`      | Electronic spatial extent                                                         | :math:`{a_0}^2`                             |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | zpve   | :math:`\textrm{ZPVE}`            | Zero point vibrational energy                                                     | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | U0     | :math:`U_0`                      | Internal energy at 0K                                                             | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | U      | :math:`U`                        | Internal energy at 298.15K                                                        | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | H      | :math:`H`                        | Enthalpy at 298.15K                                                               | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | G      | :math:`G`                        | Free energy at 298.15K                                                            | :math:`\textrm{eV}`                         |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+
        | Cv     | :math:`c_{\textrm{v}}`           | Heat capavity at 298.15K                                                          | :math:`\frac{\textrm{cal}}{\textrm{mol K}}` |
        +--------+----------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------+

        '''
        self.npz_path = npz_path # downloadable from via code as in https://github.com/dmlc/dgl/blob/3edc19528cc5b893a35c9ad2f1a6790e98b9dd95/python/dgl/data/utils.py#L110
        self.data_dict = np.load(npz_path, allow_pickle=True)
        self.label_keys = list(["mu","alpha","homo","lumo","gap","r2","zpve","U0","U","H","G","Cv"]) # all labels in R
        self.standardize_targets = standardize_targets

        self.N = self.data_dict['N'] # n nodi in g
        self.R = self.data_dict['R'] # 3d coords
        self.Z = self.data_dict['Z'] # atom number

        self.label = np.stack([self.data_dict[key] for key in self.label_keys], axis=1) # shape: (n_mols: 130831, n_labels: 12)
        if standardize_targets:
            self.label = self._standardize_targets(self.label)

        self.N_cumsum = np.concatenate([[0], np.cumsum(self.N)])

        # Create a dictionary for mapping
        # TODO forward_mapping/inverse_mapping have to be computed on the fly, here are ad-hoc for qm9
        self.forward_mapping = {1: 0, 6: 1, 7: 2, 8: 3, 9: 4} # atoms: {'H':0, 'C':1, 'N':2, 'O':3, 'F':4}
        self.inverse_mapping = {v:k for k,v in self.forward_mapping.items()}


    @property
    def labels_ordering(self):
        return self.label_keys

    @property
    def describe(self):
        data = self.label
        # Calculate descriptive statistics
        min_vals = np.min(data, axis=0)
        max_vals = np.max(data, axis=0)
        mean_vals = np.mean(data, axis=0)
        std_vals = np.std(data, axis=0)
        median_vals = np.median(data, axis=0)
        var_vals = np.var(data, axis=0)
        percentile_25_vals = np.percentile(data, 25, axis=0)
        percentile_75_vals = np.percentile(data, 75, axis=0)

        # Print the results
        print("Descriptive statistics for each column:")
        for i in range(data.shape[1]):
            print(f"Column {self.label_keys[i]}:")
            print(f"  Min: {min_vals[i]}")
            print(f"  Max: {max_vals[i]}")
            print(f"  Mean: {mean_vals[i]}")
            print(f"  Std Dev: {std_vals[i]}")
            print(f"  Median: {median_vals[i]}")
            print(f"  Variance: {var_vals[i]}")
            print(f"  25th Percentile: {percentile_25_vals[i]}")
            print(f"  75th Percentile: {percentile_75_vals[i]}")
            print()  # Blank line for readability

    def save_npz(self, folder_name = None, N = None):

        assert folder_name, f"{folder_name} must be provided"

        N = N if N else self.__len__()
        for idx in range(N):
            mol = self.__getitem__(idx)
            file = f'{folder_name}/mol_{idx}'
            # these are the kwords used in the npz
            # - coords
            # - atom_types
            # - graph_labels
            np.savez(file, coords=mol['pos'], atom_types=mol['atom_types'], graph_labels=mol['label'])

    @property
    def avg_num_atoms(self):
        return sum(self.N)/len(self.N)

    @property
    def atom_types(self):
        return set(self.Z)

    @property
    def unique_atom_types(self):
        return len(self.atom_types)

    @property
    def max_num_nodes(self):
        return max(self.N )

    def __len__(self):
        r"""Number of graphs in the dataset.

        Return
        -------
        int
        """
        return self.label.shape[0]

    def map_values(self, arr, map):
        '''
        shifts atoming nubmers wrt map=self.mapping
        invertible via map=self.inverse_mapping
        eg
        dset = instance of self
        inpt = dset[0]['atom_types'] # array([6, 1, 1, 1, 1])
        out = dset.map_values(inpt, dset.forward_mapping) # array([1, 0, 0, 0, 0])
        reverted = dset.map_values(out, dset.inverse_mapping) # array([6, 1, 1, 1, 1])
        '''

        # Use np.vectorize to apply the mapping
        vectorized_map = np.vectorize(map.get)

        # Apply the vectorized mapping function to the array
        return vectorized_map(arr)

    def get_means_and_stds(self):
        assert not self.standardize_targets
        np.set_printoptions(suppress=True, precision=5)

        return {
            'means': np.mean(self.label, axis=0),
            'stds' : np.std(self.label,  axis=0),
        }


    def _standardize_targets(self, arr):

        assert self.standardize_targets

        # Calculate the mean and standard deviation for each column
        means = np.mean(arr, axis=0)
        stds  = np.std(arr,  axis=0)

        # Standardize the columns
        standardized_arr = (arr - means) / stds

        return standardized_arr

    def __getitem__(self, idx):
        r""" Get graph and label by index

        Parameters
        ----------
        idx : int
            Item index

        Returns
        -------

            The graph contains:

            - ``ndata['R']``: the coordinates of each atom
            - ``ndata['Z']``: the atomic number

        Tensor
            Property values of molecular graphs
        """
        label = np.array(self.label[idx], dtype=np.dtype(np.float32))
        label = np.expand_dims(label, axis=0)

        R = self.R[self.N_cumsum[idx]:self.N_cumsum[idx + 1]]
        R_ = np.array(R, dtype=np.dtype(np.float32))
        R_ = np.expand_dims(R_, axis=0)

        Z_ = np.array(self.Z[self.N_cumsum[idx]:self.N_cumsum[idx + 1]], dtype=np.dtype(np.int32))
        Z_out = self.map_values(Z_, self.forward_mapping)
        assert all(Z_ == self.map_values(Z_out, self.inverse_mapping))

        return {'pos': R_, 'atom_types':Z_out, 'label':label} # look at save_npz for naming

In [29]:
dset = QM9NpzWriter(npz_path = "/storage_common/nobilm/qm_npz/qm9_eV.npz")
dset.avg_num_atoms

18.02485649425595

In [3]:
dset.labels_ordering

['mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

In [16]:
dset.get_means_and_stds()

{'means': array([   2.68217,   75.27049,   -6.53506,    0.32208,    6.85714,
        1189.45373,    4.05464,  -76.07714,  -76.54132,  -76.97889,
         -70.79997,   31.61686]),
 'stds': array([  1.49739,   8.16319,   0.59892,   1.27434,   1.28578, 280.20662,
          0.90042,  10.31253,  10.40401,  10.47797,   9.4881 ,   4.05487])}

In [18]:
dset.describe

Descriptive statistics for each column:
Column mu:
  Min: 0.0
  Max: 29.5564
  Mean: 2.6821669611942323
  Std Dev: 1.4973867265852865
  Median: 2.4818
  Variance: 2.2421670089537997
  25th Percentile: 1.5803
  75th Percentile: 3.6121

Column alpha:
  Min: 6.31
  Max: 196.62
  Mean: 75.2704936138988
  Std Dev: 8.163193599559367
  Median: 75.58
  Variance: 66.63772974388702
  25th Percentile: 70.49
  75th Percentile: 80.58

Column homo:
  Min: -11.6628000500438
  Max: -2.7673979586781488
  Mean: -6.535062266032155
  Std Dev: 0.5989174846850798
  Median: -6.560665170474943
  Variance: 0.35870215346150275
  25th Percentile: -6.873596109755166
  75th Percentile: -6.225965122375225

Column lumo:
  Min: -4.761992554264268
  Max: 5.265403195715062
  Mean: 0.3220839659190958
  Std Dev: 1.2743437525787005
  Median: 0.345584602509464
  Variance: 1.623951999736364
  25th Percentile: -0.63130415576532
  75th Percentile: 1.3796172714354193

Column gap:
  Min: 0.6694000961994342
  Max: 16.92820324575

In [20]:
folder = "/storage_common/nobilm/qm_npz/TEST_FULL_LABELS"
dset.save_npz(folder, 100)

In [17]:
dset.unique_atom_types, dset.atom_types
# {H 1, C 6, N 7, O 8, F 9})

(5, {1, 6, 7, 8, 9})

In [1]:
from geqtrain.scripts.evaluate import load_model
from pathlib import Path

model, config = load_model(Path('/home/nobilm@usi.ch/GEqTrain/results/qm9_production_run/first_exp/best_model.pth'))

/home/nobilm@usi.ch/miniconda3/envs/geqtrain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nobilm@usi.ch/miniconda3/envs/geqtrain/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
INFO:geqtrain-evaluate:loaded model from training session.


In [3]:
for n,v in model.named_parameters():
    if n == "output_head.bias_":
        print(n, v)

output_head.bias_ Parameter containing:
tensor([ 2.6675e+00,  7.4217e+01, -6.6978e+00,  1.0421e-04,  6.6705e+00,
         1.1883e+03,  2.6190e+00, -7.4520e+01, -7.4951e+01, -7.5494e+01,
        -6.9551e+01,  3.1630e+01], requires_grad=True)


# Evaluation

In [5]:
import numpy as np
ds = np.load("/storage_common/nobilm/qm_npz/test/mol_100000.npz", allow_pickle=True)

In [7]:
import torch
from geqtrain.data import AtomicDataDict
def prepare_dataset(dataset, r_max):
    coords = torch.from_numpy(dataset.get("coords"))
    node_types = torch.from_numpy(dataset.get("atom_types"))
    batch = torch.zeros(coords.shape[-2], dtype=torch.long)

    return [{
        AtomicDataDict.POSITIONS_KEY: pos,
        f"{AtomicDataDict.POSITIONS_KEY}_slices": torch.tensor([0, len(pos)]),
        AtomicDataDict.NODE_TYPE_KEY: node_types,
        AtomicDataDict.EDGE_INDEX_KEY: get_edge_index(positions=pos, r_max=r_max),
        AtomicDataDict.BATCH_KEY: batch,
    }
    for pos in coords
    ], dataset

def get_edge_index(positions: torch.Tensor, r_max: float):
    dist_matrix = torch.norm(positions[:, None, ...] - positions[None, ...], dim=-1).fill_diagonal_(torch.inf)
    return torch.argwhere(dist_matrix <= r_max).T.long()

In [9]:
batch, _= prepare_dataset(ds, r_max=50.0 )

In [17]:
with torch.no_grad():
    out = model(batch[0])

In [46]:
mae_res = abs(out['graph_labels'] - ds['graph_labels'])
for idx, k in enumerate(dset.labels_ordering):
    if k in ['mu', 'homo', 'lumo', 'gap', 'zpve', 'U0', 'U', 'H', 'G']:
        print(k, mae_res[:,idx].item() *1000)
    else:
        print(k, mae_res[:,idx].item())

mu 463.5958671569824
alpha 1.5829849243164062
homo 281.6300392150879
lumo 337.59403228759766
gap 70.78933715820312
r2 3.9703369140625
zpve 4.011631011962891
U0 644.47021484375
U 655.9829711914062
H 654.9530029296875
G 652.9464721679688
Cv 0.4727668762207031
